In [1]:
import sys
import os

# Add the `src` folder to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))




In [2]:
%pwd

'/Users/srujanappu/Text-Summarizer-Project/research'

In [3]:
os. chdir("../")

In [4]:
%pwd

'/Users/srujanappu/Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories






In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile

from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [9]:


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with headers: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        """
        Extracts the downloaded zip file into the specified directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-03-27 06:54:31,685: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-03-27 06:54:31,687: INFO: common: YAML file: params.yaml loaded successfully]
[2025-03-27 06:54:31,687: INFO: common: Created directory at: artifacts]
[2025-03-27 06:54:31,688: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-03-27 06:54:34,000: INFO: 622720954: artifacts/data_ingestion/data.zip downloaded with headers: 
Connection: close
Content-Length: 8275776
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "380137e68d51d0343f9005ea6d1853f00551f687a83b1b9f52d686db72657e20"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0E7A:3673CD:19E779C:2039AB3:67E52E67
Accept-Ranges: bytes
Date: Thu, 27 Mar 2025 10:54:32 GMT
Via: 1.1 varnish
X-Served-By: cache-yyz4579-YYZ
X-Cache: MISS
X-